In [1]:
from langchain_community.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.1)

def initialize_retriever(file_path):
    cache_dir = LocalFileStore("./.cache/")
    splitter = CharacterTextSplitter.from_tiktoken_encoder(separator="\n", chunk_size=1000, chunk_overlap=100)

    docs = UnstructuredFileLoader(file_path).load_and_split(text_splitter=splitter)
    embeddings = CacheBackedEmbeddings.from_bytes_store(OpenAIEmbeddings(), cache_dir)
    return FAISS.from_documents(docs, embeddings).as_retriever()

retriever = initialize_retriever("./files/1984_chapter_three.txt")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

def load_memory(_):
    return memory.load_memory_variables({}).get("chat_history", [])


map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system",  
        """
        Use the following context of a long document to see if any of the text is relevant to answer the question.
        Return any relevant text verbatim
        -------
        {context}
        """ 
    ),
    ("human", "{question}")
])
    
map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
    documents = inputs["documents"]
    return "\n\n".join(doc.page_content for doc in documents)

map_chain = {
    "documents": retriever, 
    "question": RunnablePassthrough()
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        """
        Given the following extracted parts of a long documents and a question, create a final answer.
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.
        -------
        {context}
        """
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

A = RunnableLambda(load_memory)
chain = {"chat_history":A, "context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm 

def invoke_chain(question):
    response = chain.invoke(question)
    memory.save_context(
        inputs={"input":question}, 
        outputs={"output":response.content}
    )
    print(response)
invoke_chain("Was Aaronson guilty of a crime?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")


/var/folders/dd/dvf_l6wd65n98d8w8p8_w6_w0000gn/T/ipykernel_35342/3483984907.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.1)
/var/folders/dd/dvf_l6wd65n98d8w8p8_w6_w0000gn/T/ipykernel_35342/3483984907.py:17: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  docs = UnstructuredFileLoader(file_path).load_and_split(text_spli

content='Yes, according to the extracted text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 3099, 'total_tokens': 3126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-599df7f8-3f63-471e-a909-e91f385e7936-0'
content='He traced with his finger in the dust on the table the message: 2+2=5.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 3564, 'total_tokens': 3586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 

In [3]:
print(memory.load_memory_variables({})["chat_history"])

[HumanMessage(content='Was Aaronson guilty of a crime?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, according to the extracted text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What message did he write in the table?', additional_kwargs={}, response_metadata={}), AIMessage(content='He traced with his finger in the dust on the table the message: 2+2=5.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Who is Julia?', additional_kwargs={}, response_metadata={}), AIMessage(content='Julia is a character in the text who is involved in a romantic relationship with the protagonist, Winston.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what was my first question?', additional_kwargs={}, response_metadata={}), AIMessage(content="Your first question was about Aaronson's guilt in a crime.", additional_kwargs={}, response_me

In [2]:
invoke_chain("what was my first question?")

content="Your first question was about Aaronson's guilt in a crime." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 3558, 'total_tokens': 3572, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-1d321c5a-a9fd-4680-aadc-6fa791796a02-0'
